In [1]:
import argparse
import collections
import sklearn.metrics as metrics
import tensorboardX as tb
import torch as th
import torch.nn.functional as F
import torch.nn.modules.loss as loss
import torch.optim as optim
import torch.utils as utils
import data
import my
import lenet
import resnet

In [2]:
args = argparse.Namespace()
args.batch_size = 50
args.gpu = 1
args.log_every = 1000
args.n_iterations = 50000

keys = sorted(vars(args).keys())
excluded = ('gpu',)
run_id = 'relabelled-ce-' + '-'.join('%s-%s' % (key, str(getattr(args, key))) for key in keys if key not in excluded)
writer = tb.SummaryWriter('runs/' + run_id)

In [3]:
if args.gpu < 0:
    cuda = False
else:
    cuda = True
    th.cuda.set_device(args.gpu)

labelling = {(0, 9) : 0, (9, 10) : 1}
# labelling = {(0, 5) : 0, (5, 6) : 1, (6, 7) : 2, (7, 8) : 3, (8, 9) : 4, (9, 10) : 5}
train_x, train_y, test_x, test_y = data.load_cifar10(labelling, rbg=True, torch=True)

train_set = utils.data.TensorDataset(train_x, train_y)
train_loader = utils.data.DataLoader(train_set, 4096, drop_last=False)
test_set = utils.data.TensorDataset(test_x, test_y)
test_loader = utils.data.DataLoader(test_set, 4096, drop_last=False)

def TrainLoader():
    dataset = utils.data.TensorDataset(train_x, train_y)
    new_loader = lambda: iter(utils.data.DataLoader(dataset, args.batch_size, shuffle=True))
    contextualize = lambda x, y: (x.cuda(), y.cuda()) if cuda else (x, y)
    while True:
        try:
            yield contextualize(*next(loader))
        except:
            loader = new_loader()
            yield contextualize(*next(loader))

loader = TrainLoader()

n_classes = int(train_y.max() - train_y.min() + 1)

In [4]:
def global_scores(c, loader):
    keys = ('accuracy', 'precision', 'recall', 'f1')
    scores = (
        metrics.accuracy_score,
        lambda y, y_bar: metrics.precision_score(y, y_bar, average='macro'),
        lambda y, y_bar: metrics.recall_score(y, y_bar, average='macro'),
        lambda y, y_bar: metrics.f1_score(y, y_bar, average='macro'),
    )
    values = [value.item() for value in my.global_scores(c, loader, scores)]
    return collections.OrderedDict(zip(keys, values))

In [5]:
# c = lenet.LeNet(3, n_classes)
c = resnet.ResNet(18, n_classes)

if cuda:
    c.cuda()
    
# optimizer = optim.SGD(c.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-4)
optimizer = optim.Adam(c.parameters())

for key, value in global_scores(c, test_loader).items():
    print(key, value)

accuracy 0.6848
precision 0.5253333333333333
recall 0.511299667988001
f1 0.48839128665436204


In [6]:
n_iterations = 0
for i in range(args.n_iterations):
    x, y = next(loader)
    ce = loss.CrossEntropyLoss()(c(x), y)
    optimizer.zero_grad()
    ce.backward()
    optimizer.step()

    if (i + 1) % args.log_every == 0:
        train_scores = global_scores(c, train_loader)
        test_scores = global_scores(c, test_loader)

        prefix = '0' * (len(str(args.n_iterations)) - len(str(i + 1)))
        print('[iteration %s%d]' % (prefix, i + 1) + \
              ' | '.join('%s %0.3f/%0.3f' % (key, value, test_scores[key]) for key, value in train_scores.items()))

        for key, value in train_scores.items():
            writer.add_scalar('train-' + key, value, i)

        for key, value in test_scores.items():
            writer.add_scalar('test-' + key, value, i)

[iteration 01000]accuracy 0.947/0.944 | precision 0.779/0.774 | recall 0.901/0.886 | f1 0.827/0.818
[iteration 02000]accuracy 0.958/0.953 | precision 0.876/0.866 | recall 0.885/0.871 | f1 0.881/0.869
[iteration 03000]accuracy 0.971/0.966 | precision 0.904/0.886 | recall 0.933/0.921 | f1 0.918/0.903
[iteration 04000]accuracy 0.977/0.971 | precision 0.940/0.914 | recall 0.934/0.922 | f1 0.937/0.918
[iteration 05000]accuracy 0.978/0.970 | precision 0.908/0.879 | recall 0.965/0.944 | f1 0.934/0.909
[iteration 06000]accuracy 0.983/0.972 | precision 0.945/0.911 | recall 0.958/0.930 | f1 0.951/0.920
[iteration 07000]accuracy 0.987/0.975 | precision 0.948/0.909 | recall 0.976/0.950 | f1 0.961/0.928
[iteration 08000]accuracy 0.988/0.974 | precision 0.952/0.903 | recall 0.979/0.948 | f1 0.965/0.924
[iteration 09000]accuracy 0.991/0.976 | precision 0.963/0.910 | recall 0.986/0.952 | f1 0.974/0.929
[iteration 10000]accuracy 0.990/0.973 | precision 0.977/0.922 | recall 0.971/0.929 | f1 0.974/0.925


In [7]:
# for pg in optimizer.param_groups:
#     pg['lr'] *= 0.1